<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/diabetic_retinopathy/KaggleOnlyDiabeticRetinopathyDomainAdaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://zenodo.org/record/3412990/files/kaggle_dr.txt.gz?download=1 -O kaggle_dr.txt.gz

--2019-09-22 03:34:49--  https://zenodo.org/record/3412990/files/kaggle_dr.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106643 (104K) [application/octet-stream]
Saving to: ‘kaggle_dr.txt.gz’

kaggle_dr.txt.gz    100%[===================>] 104.14K  --.-KB/s    in 0.06s   

2019-09-22 03:34:50 (1.70 MB/s) - ‘kaggle_dr.txt.gz’ saved [106643/106643]



In [0]:
import gzip
import numpy as np

orig_kaggle_onehot_labels = [] #5 classes
orig_kaggle_predlogits = []
for row in gzip.open("kaggle_dr.txt.gz"):
    _, kaggle_label, logits = row.decode("utf-8").rstrip().split("\t")
    kaggle_label = int(kaggle_label)
    logits = [float(x) for x in logits.split(",")]
    labelsvec = np.zeros(5)
    labelsvec[kaggle_label] = 1.0
    orig_kaggle_onehot_labels.append(labelsvec)
    orig_kaggle_predlogits.append(logits)
orig_kaggle_onehot_labels = np.array(orig_kaggle_onehot_labels)
orig_kaggle_predlogits = np.array(orig_kaggle_predlogits)

In [0]:
from sklearn.metrics import roc_auc_score
from scipy.special import softmax

print(roc_auc_score(y_true=1-orig_kaggle_onehot_labels[:,0],
                    y_score=1-softmax(orig_kaggle_predlogits, axis=-1)[:,0]))

0.8945151902722942


In [0]:
![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd /content/abstention
!git pull
!pip uninstall abstention
!pip install .
%cd ..

/content/abstention
Already up to date.
Uninstalling abstention-0.1.3.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/abstention-0.1.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/abstention/*
Proceed (y/n)? y
  Successfully uninstalled abstention-0.1.3.0
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.3.0-cp36-none-any.whl size=45713 sha256=bd982733f1307b42afcf218546d07bf8c72da0d755d29f6d69cbefe0be41be84
  Stored in directory: /tmp/pip-ephem-wheel-cache-9e4zkkqk/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
/content


In [0]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall labelshiftexperiments
!pip install .
%cd ..

fatal: destination path 'labelshiftexperiments' already exists and is not an empty directory.
/content/labelshiftexperiments
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/blindauth/labelshiftexperiments
   22144b7..a437953  master     -> origin/master
Updating 22144b7..a437953
Fast-forward
 labelshiftexperiments/cifarandmnist.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
Uninstalling labelshiftexperiments-0.1.0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/labelshiftexperiments-0.1.0.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/labelshiftexperiments/*
Proceed (y/n)? y
  Successfully uninstalled labelshiftexperiments-0.1.0.0
Processing /content/labelshiftexperiments
  Created wheel for labelshiftexperiments: filename=labelshiftexperiments-0.1.

In [0]:
#do calibration and domain adaptation
from collections import OrderedDict
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
from abstention.calibration import (
    TempScaling, CrossValidatedBCTS, softmax)
from abstention.label_shift import (EMImbalanceAdapter, BBSEImbalanceAdapter)

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions=[0,1,2,3,4])),
    #('CVBCTS', CrossValidatedBCTS(verbose=True,
    #                              num_crossvalidation_splits=5,
    #                              betas_to_try=[0.0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]
    #                             ))
])

adaptname_to_adapter = OrderedDict([
    ('em', EMImbalanceAdapter()),
    ('bbse-hard', BBSEImbalanceAdapter(soft=False)),
    ('bbse-soft', BBSEImbalanceAdapter(soft=True))
])
    
adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'BCTS'),
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'BCTS'),
    #('em', 'CVBCTS'),
]
    

In [0]:
len(orig_kaggle_onehot_labels)

3514

In [0]:
np.mean(orig_kaggle_onehot_labels, axis=0)

array([0.7310757 , 0.07057484, 0.15253273, 0.02447353, 0.0213432 ])

In [0]:
from collections import defaultdict
import numpy as np
import sys
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
reload(abstention.figure_making_utils)
from abstention.calibration import (
    TempScaling, #CrossValidatedBCTS,
    ConfusionMatrix, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import numpy as np
from collections import defaultdict, OrderedDict
import scipy

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist

metric_to_adaptname_to_perfs = defaultdict(lambda: defaultdict(list))

num_trials = 100
tweakone_alphas_and_samplesize = [(0.5,500), (0.9,500),
                                  (0.5,1000), (0.9,1000),
                                  (0.5,1500), (0.9,1500)]

tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals = (
        defaultdict(
            lambda: defaultdict(
                     lambda: defaultdict(
                              lambda: defaultdict(list)))))
tweakone_alpha_to_samplesize_to_baselineperf = defaultdict(lambda: defaultdict(list))
metric_to_samplesize_to_calibname_to_unshiftedvals = defaultdict(
    lambda: defaultdict(lambda: defaultdict(list)))
samplesizesseen = set()

orig_class_priors = np.mean(orig_kaggle_onehot_labels, axis=0)
kappaweights = np.square(np.arange(5)[:,None] - np.arange(5)[None,:])

for (alpha, samplesize) in tweakone_alphas_and_samplesize:
    for trail_num in range(num_trials):
        rng = np.random.RandomState(trail_num*10)
        #sample a split of pseudovalid/pseudotest indices
        indices = rng.choice(np.arange(len(orig_kaggle_onehot_labels)),
                             size=len(orig_kaggle_onehot_labels),
                             replace=False)
        pseudovalid_indices = indices[:samplesize]
        pseudotest_indices = indices[samplesize:]
        assert ((len(pseudovalid_indices)+len(pseudotest_indices))
                ==len(orig_kaggle_onehot_labels))
        
        pseudovalid_preacts = orig_kaggle_predlogits[pseudovalid_indices]
        pseudotest_preacts = orig_kaggle_predlogits[pseudotest_indices]
        pseudovalid_labels = orig_kaggle_onehot_labels[pseudovalid_indices]
        pseudotest_labels = orig_kaggle_onehot_labels[pseudotest_indices]
        
        draw_test_indices = cifarandmnist.get_func_to_draw_label_proportions(
                              test_labels=pseudotest_labels)
        
        #perform calibration
        (calibname_to_calibfunc,
         calibname_to_calibpseudovalidpreds) = (
                    cifarandmnist.run_calibmethods(
                       valid_preacts=pseudovalid_preacts,
                       valid_labels=pseudovalid_labels,
                       test_preacts=pseudotest_preacts,
                       test_labels=pseudotest_labels,
                       calibname_to_calibfactory=
                        calibname_to_calibfactory,
                       samplesize=samplesize,
                       samplesizesseen=samplesizesseen,
                       metric_to_samplesize_to_calibname_to_unshiftedvals=
                        metric_to_samplesize_to_calibname_to_unshiftedvals))
        
        #perform the knockout shift
        altered_class_priors = np.zeros(5)
        altered_class_priors[0] = alpha
        altered_class_priors[1:] = (1-alpha)*orig_class_priors[1:]
        
        pseudotest_indices = draw_test_indices(
                          total_to_return=samplesize,                    
                          label_proportions=altered_class_priors,        
                          rng=rng)
        shifted_pseudotest_labels = pseudotest_labels[pseudotest_indices]
        shifted_pseudotest_preacts = pseudotest_preacts[pseudotest_indices]
        
        calibname_to_calibshiftedtestpreds = {}
        for (calibname, calibfunc) in calibname_to_calibfunc.items():
            calibname_to_calibshiftedtestpreds[calibname] =(
                calibfunc(shifted_pseudotest_preacts))
        
        shifted_pseudotest_baseline_perf =\
          abstention.abstention.weighted_kappa_metric(
            predprobs=abstention.calibration.softmax(
                        preact=shifted_pseudotest_preacts,
                        temp=1.0, biases=None),
            true_labels=shifted_pseudotest_labels,
            weights=kappaweights, mode='argmax')
        tweakone_alpha_to_samplesize_to_baselineperf[alpha][samplesize].append(
            shifted_pseudotest_baseline_perf)
        true_shifted_priors = np.mean(shifted_pseudotest_labels, axis=0)
        
        for adapter_name,calib_name in adaptncalib_pairs:
            calib_shifted_pseudotest_preds =\
              calibname_to_calibshiftedtestpreds[calib_name]
            calib_pseudovalid_preds =\
              calibname_to_calibpseudovalidpreds[calib_name]
            imbalance_adapter = adaptname_to_adapter[adapter_name]
            imbalance_adapter_func = imbalance_adapter(
                valid_labels=pseudovalid_labels,                       
                tofit_initial_posterior_probs=calib_shifted_pseudotest_preds, 
                valid_posterior_probs=calib_pseudovalid_preds)
            adapted_shifted_pseudotest_preds = imbalance_adapter_func(        
                        calib_shifted_pseudotest_preds)
            estim_shifted_priors = np.mean(adapted_shifted_pseudotest_preds,  
                                           axis=0)
            #compute metrics for quality of shift
            adapted_shifted_pseudotest_perf =\
              abstention.abstention.weighted_kappa_metric(
                predprobs=adapted_shifted_pseudotest_preds,
                true_labels=shifted_pseudotest_labels,
                weights=kappaweights, mode='argmax')
            delta_from_baseline = (adapted_shifted_pseudotest_perf-
                                   shifted_pseudotest_baseline_perf)
            
            tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                alpha][samplesize][adapter_name+":"+calib_name][
                'jsdiv'].append(
                  scipy.spatial.distance.jensenshannon(
                      p=true_shifted_priors, q=estim_shifted_priors))
            tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                alpha][samplesize][adapter_name+":"+calib_name][
                'delta_perf'].append(delta_from_baseline)
        
    if (samplesize not in samplesizesseen):
        print("Calibration stats")
        for metric in ['ece', 'nll', 'jsdiv']:
            print("Metric",metric)
            for calibname in calibname_to_calibfactory:
                print(calibname, np.mean(                     
                       metric_to_samplesize_to_calibname_to_unshiftedvals[
                        metric][samplesize][calibname]))  
        samplesizesseen.add(samplesize)

    print("On alpha",alpha,"sample size", samplesize)
    for metric_name in ['delta_perf', 'jsdiv']: 
        print("Metric",metric_name)
        for adapter_name,calib_name in adaptncalib_pairs:
            adaptncalib_name = adapter_name+":"+calib_name
            n = len(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                  alpha][samplesize][adaptncalib_name][metric_name])

            print(adaptncalib_name, np.mean(
             tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
               alpha][samplesize][adaptncalib_name][metric_name]), "+/-",
             (1.0/np.sqrt(n))*np.std(
             tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
               alpha][samplesize][adaptncalib_name][metric_name],
               ddof=1))           
            sys.stdout.flush()

Calibration stats
Metric ece
None 8.733662753798031
TS 3.649580871054098
BCTS 2.2547945348797995
Metric nll
None 0.6402319025011183
TS 0.5711691961104738
BCTS 0.5144445766964391
Metric jsdiv
None 0.07661477273081052
TS 0.09092214616308848
BCTS 0.02565906567873681
On alpha 0.5 sample size 500
Metric delta_perf
bbse-hard:None 0.0036097804004039426 +/- 0.0038464193927202194
bbse-soft:None 0.013992563092023689 +/- 0.0023997638799342714
bbse-soft:TS 0.011041795059002544 +/- 0.00219241347775606
bbse-soft:BCTS 0.0016454834167431765 +/- 0.003071274082933666
em:None 0.021033992877224135 +/- 0.0016896370051703719
em:TS 0.01815933092940432 +/- 0.0019622088088127126
em:BCTS 0.008566303260036734 +/- 0.002178356124912753
Metric jsdiv
bbse-hard:None 0.2041263856630507 +/- 0.012498901488122827
bbse-soft:None 0.1660728470328038 +/- 0.0082909609681407
bbse-soft:TS 0.1843171877641111 +/- 0.008061539221192763
bbse-soft:BCTS 0.15304000011417032 +/- 0.008238813194105615
em:None 0.10749938801258889 +/- 0.003

In [0]:
import json
import os
file_out = "label_shift_adaptation_results.json"
dict_to_write = {
    "tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "tweakone_alpha_to_samplesize_to_baselineperf":
     tweakone_alpha_to_samplesize_to_baselineperf,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [0]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("label_shift_adaptation_results.json.gz").read())
tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
tweakone_alpha_to_samplesize_to_baselineperf =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_baselineperf']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [0]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table

metricname_to_nicename = {'nll': 'nll', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = num_trials
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'jsdiv', 'ece'],
        samplesizes_in_table=['500', '1000', '1500'],
        calibnames_in_table=['None', 'TS', 'BCTS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="KaggleDR Calibration metric differences", label="kaggledrcalibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c c c | c c c | c c c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{nll} & \multicolumn{3}{| c}{jsdiv} & \multicolumn{3}{| c}{ECE}\\
    \cline{2-10}
    & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    None & 0.64; 2.0 & 0.639; 2.0 & 0.639; 2.0 & 0.077; 1.0 & 0.077; 1.0 & 0.077; 1.0 & 8.734; 2.0 & 8.737; 2.0 & 8.767; 2.0\\
    TS & 0.571; 1.0 & 0.57; 1.0 & 0.569; 1.0 & 0.091; 2.0 & 0.091; 2.0 & 0.091; 2.0 & 3.65; 0.91 & 3.729; 0.95 & 3.853; 0.92\\
    BCTS & \textbf{0.514; 0.0} & \textbf{0.511; 0.0} & \textbf{0.511; 0.0} & \textbf{0.026; 0.0} & \textbf{0.02; 0.0} & \textbf{0.018; 0.0} & \textbf{2.255; 0.09} & \textbf{2.097; 0.05} & \textbf{2.171; 0.08}\\
  \end{tabular}}
  \caption{KaggleDR Calibration metric differences}
  \label{tab:cifar10calibrationcomparison}
\e

In [0]:
from labelshiftexperiments.maketable import render_adaptation_table
    
methodgroups = OrderedDict([
  ('em', ['em:None', 'em:TS', 'em:BCTS']),
  ('bbse', ['bbse-hard:None', 'bbse-soft:None',
           'bbse-soft:TS', 'bbse-soft:BCTS'])])
metric_to_largerisbetter = {'perf':True}
samplesizes_in_table = ['500', '1000', '1500']
adaptname_to_nicename = {'em': 'EM',
                           'bbse-soft': 'BBSE-soft',
                           'bbse-hard': 'BBSE-hard'}
calibname_to_nicename = {'None': 'None',
                           'TS': 'TS',
                           'BCTS': 'BCTS'}

tweakone_alphas_in_table = ['0.5', '0.9']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR Metric: JS Divergence, tweakone shift",
    label="kaggledrjsdivtweakoneshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_perf',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR Metric: $\\Delta$\\%Accuracy, tweakone shift",
    label="kaggledrdeltaacctweakoneshift",
    applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.5$} & \multicolumn{3}{| c}{$\alpha=0.9$}\\ 
    \cline{3-8}
    & & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    \hline
    EM & None & 0.107; 0.78 & 0.102; 1.03 & 0.095; 1.06 & 0.098; 0.85 & 0.087; 0.84 & 0.087; 0.82\\
    EM & TS & 0.126; 1.19 & 0.108; 1.05 & 0.101; 1.11 & 0.133; 1.88 & 0.122; 1.94 & 0.123; 1.95\\
    EM & BCTS & 0.111; 1.03 & 0.095; 0.92 & \textbf{0.082; 0.83} & \textbf{0.078; 0.27} & \textbf{0.065; 0.22} & \textbf{0.063; 0.23}\\
    \hline
    \hline
    BBSE-hard & None & 0.204; 1.78 & 0.186; 2.26 & 0.153; 2.09 & 0.151; 2.12 & 0.137; 2.27 & 0.114; 2.19\\
    BBSE-soft & None & 0.166; 1.13 & 0.139; 1.43 & 0.12; 1.31 & 